# Napari image viewer

Uses the octopuslite dask image loader to automatically sort and load images from multichannel time lapse experiments. Can also use btrack to load associated tracks and segmentation images.

In [1]:
import napari
from octopuslite import DaskOctopusLiteLoader
import glob, os
import numpy as np
from datetime import datetime
from skimage.io import imread
from tqdm.auto import tqdm
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def compute(dask_image_stack, N = None):
    
    """lazy hack to get around dask not loading transformed stacks in one go"""
    
    if N == None:
        N = (0, len(dask_image_stack) )
        
    return np.stack([frame.compute().astype(np.uint8) for frame in tqdm(dask_image_stack[N[0]:N[1]], total = N[1]-N[0])], axis = 0)

In [2]:
### load experiment information
expt_info = pd.read_csv('extrusion_expts.csv', )
del expt_info['Unnamed: 0']
expt_info

,EXP n˚,POSITION,CELL TYPE
0,ND0013,Pos3,90:10 wt:ras+
1,ND0013,Pos4,90:10 wt:ras+
2,ND0013,Pos5,90:10 wt:ras+
3,ND0013,Pos6,90:10 wt:ras+
4,ND0013,Pos7,90:10 wt:ras+
...,...,...,...
101,ND0025,Pos9,97.5:2.5 wt:ras+
102,ND0025,Pos10,97.5:2.5 wt:ras+
103,ND0025,Pos11,97.5:2.5 wt:ras+
104,ND0025,Pos12,97.5:2.5 wt:ras+


In [3]:
### Set root directory (server or local)
root_dir = '/home/nathan/data/kraken/ras/'#'/media/nathan/Transcend/Nathan/data/kraken/ras'
### make output directory for .csv list of extrusion coordinates
os.makedirs('extrusions', exist_ok = True)

In [4]:
### exclude certain expts if you wish
expt_info = expt_info[((expt_info['CELL TYPE'] == '99:1 wt:ras+'))]# & (expt_info['POSITION'] == 'Pos3')& (expt_info['POSITION'] == 'Pos4'))]

In [ ]:
### iterate over each competitive experiment
for i, expt_pos in tqdm(expt_info.iterrows(), desc = 'Progress throughout valid expts', total = len(expt_info)):
    ### get expt info
    expt = expt_pos['EXP n˚']
    pos = expt_pos['POSITION']
    print(f'Loading experiment {expt} {pos}')
    ### load images using dask
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    ### check if already done this experiment
    fn = f'extrusions/{expt}_{pos}_extrusions.csv'
    if os.path.exists(fn):
        date_time = datetime.now().strftime("%m.%d.%Y.%H:%M:%S")
        fn = f'extrusions/{expt}_{pos}_extrusions_updated_{date_time}.csv'
        continue ### comment out this line if you dont want to skip if already exists
    transform_path = f'{root_dir}/{expt}/{pos}/transform_tensor.npy'
    if not os.path.exists(transform_path):
        print('No transformation tensor exists, skipping experiment')
        continue
    images = DaskOctopusLiteLoader(image_path, transforms=transform_path)
    ### (optional) load images into memory for quicker scrolling
    stack_gfp = images['gfp']#compute(images['gfp'])
    stack_rfp = images['rfp']#compute(images['rfp'])
    ### open napari
    viewer = napari.Viewer()
    ### add images to napari
    viewer.add_image(stack_gfp, name=f"gfp {expt}, {pos}", 
                     contrast_limits=[0,119],
                     blending = 'additive', colormap = 'green')
#     viewer.add_image(stack_bf, name=f"bf {expt}, {pos}", 
#                      contrast_limits=[0,255],
#                      blending = 'additive')
    viewer.add_image(stack_rfp, name=f"rfp {expt}, {pos}", 
                     contrast_limits=[0,55], 
                     blending = 'additive', colormap = 'magenta')
    ### add point layer for labelling extrusion
    points_layer = viewer.add_points(np.empty((0,3)), ndim = 3,face_color='orange', name = 'extrusions', out_of_slice_display=True)
    ### open at start of time lapse
    viewer.dims.current_step = (0,0,0)
    ### only open next experiment after closing current one
    viewer.show(block=True)
    ### save out list of extrusion coords
    points_layer.save(fn)